In [1]:
import torch
import torchvision
#for reproducibility
torch.manual_seed(0)
import numpy as np 
np.random.seed(0)
!nvcc --version
from qtorch_plus.quant import Quantizer, quantizer
from qtorch_plus.optim import OptimLP
from torch.optim import SGD
from qtorch_plus import FloatingPoint, Posit

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cpu/build.ninja...
Building extension module quant_cpu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module quant_cpu...
Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...


ninja: no work to do.


Detected CUDA files, patching ldflags
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cuda/build.ninja...
/home/yg9bq/miniconda3/envs/posit/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module quant_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module quant_cuda...


In [2]:
## Create dataloader, in PyTorch, we feed the trainer data with use of dataloader
## We create dataloader with dataset from torchvision, 
## and we dont have to download it seperately, all automatically done

# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 64 # We use a small batch size here for training
batch_size_test = 1024 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
#image datasets
train_dataset = torchvision.datasets.MNIST('/localsdd/yg9bq/LLM_quantization/mnist-data/', 
                                           train=True, 
                                           download=True,
                                           transform=image_transform)
test_dataset = torchvision.datasets.MNIST('/localsdd/yg9bq/LLM_quantization/mnist-data/', 
                                          train=False, 
                                          download=True,
                                          transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True)

# define two floating point formats
bit_8 = Posit(nsize=8, es=2, scale = 1.0) #scale to be multiplied with the number. In the paper e.g. exponent bias = 5 => scale = 32.0 . Default value of scale = 1.0 (no scaling)
bit_16 = Posit(nsize=16, es=2, scale = 1.0)

# define quantization functions
weight_quant = quantizer(forward_number=bit_8,
                        forward_rounding="nearest")
grad_quant = quantizer(forward_number=bit_8,
                        forward_rounding="nearest")
momentum_quant = quantizer(forward_number=bit_16,
                        forward_rounding="nearest")
acc_quant = quantizer(forward_number=bit_16,
                        forward_rounding="nearest")

# define a lambda function so that the Quantizer module can be duplicated easily
act_error_quant = lambda : Quantizer(forward_number=bit_8, backward_number=bit_8,
                        forward_rounding="nearest", backward_rounding="nearest")

In [3]:
## Now we can start to build our CNN model
## We first import the pytorch nn module and optimizer
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## Then define the model class
class CNN(nn.Module):
    def __init__(self,quant):
        super(CNN, self).__init__()
        #input channel 1, output channel 10
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        #input channel 10, output channel 20
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        #dropout layer
        #fully connected layer
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.quant = quant()
    def forward(self, x):
        x = self.conv1(x)
        x = self.quant(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.quant(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = self.quant(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.quant(x)
        return F.log_softmax(x,dim=0)

In [4]:
## create model and optimizer
learning_rate = 0.01
momentum = 0.5
device = "cuda"
model = CNN(act_error_quant).to(device) #using gpu here
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)
optimizer = OptimLP(optimizer,
                    weight_quant=weight_quant,
                    grad_quant=grad_quant,
                    momentum_quant=momentum_quant,
                    acc_quant=acc_quant,
                    grad_scaling=1 # do loss scaling if necessary
)

In [8]:
# from tqdm import tqdm_notebook as tqdm
# from tqdm.notebook import tqdm

##define train function
## Define train function without tqdm
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    counter = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        counter += 1
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            
## Define test function (no changes needed)
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [9]:
num_epoch = 5
for epoch in range(1, num_epoch + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 4.163303

Test set: Average loss: 4.9032, Accuracy: 9618/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.318387

Test set: Average loss: 4.8016, Accuracy: 9720/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.160347

Test set: Average loss: 4.7677, Accuracy: 9755/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.021927

Test set: Average loss: 4.7364, Accuracy: 9806/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 1.966524

Test set: Average loss: 4.7238, Accuracy: 9832/10000 (98%)

